# Mini-GPT
### A mini GPT built using PyTorch and pre-trained on the text of F. Scott Fitzgerald's The Great Gatsby, obtained from Project Gutenberg:
https://www.gutenberg.org/ebooks/64317

Self-Attention Transformer Architecture:
"Attention is all You Need": https://arxiv.org/abs/1706.03762


In [ ]:
# !pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00


In [ ]:
# from datasets import load_dataset

# openwebtext_corpus = load_dataset("Skylion007/openwebtext")
# openwebtext_corpus = openwebtext_corpus.shuffle(seed = 42)
# openwebtext_corpus_split = openwebtext_corpus['train'].train_test_split(test_size=0.2)

# tokenized_corpus = openwebtext_corpus_split['train'].map(lambda x: {'text': x['text'].replace('\n', '')})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `dataset

Generating train split:   0%|          | 0/8013769 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
# from tqdm.auto import tqdm  # for our loading bar

# chars = []
# file_count = 0

# for sample in tqdm(openwebtext_corpus_split['train']):
#     # remove newline characters from each sample as we need to use exclusively as seperators
#     sample = sample['text'].replace('\n', '')
#     chars.extend(sorted(list(set(sample))))
# vocab_size = len(chars)
# with open(f'./content/vocab.txt', 'w', encoding='utf-8') as fp:
#     fp.write('\n'.join(chars))

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7212392
    })
    test: Dataset({
        features: ['text'],
        num_rows: 801377
    })
})

## Make Imports and set Global Model Parameters


In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_interval = 100
n_embd = 384 # total no. of dimensions we want to capture from all the concatenated heads
n_head = 8
n_layer = 4 #no of decoder blocks
dropout = 0.2


cuda


## File Import

In [2]:
gatsbyfile = '/content/the_great_gatsby.txt'
chars = ""
with open(gatsbyfile, "r", encoding = 'utf-8') as f:
  text = f.read()
  text = text.replace('\n', '')
  chars = sorted(list(set(text)))
vocab_size = len(chars)

## Encoding text into tokens

In [3]:
string_to_int = { character:index for index, character in enumerate(chars) }
int_to_string =  { index:character for index, character in enumerate(chars) }

encode = lambda string: [string_to_int[c] for c in string]
decode = lambda integer: ''.join([int_to_string[i] for i in integer])

encoded_string = encode(text)
decoded_string = decode(encoded_string)
# print(encoded_string)
print(decoded_string)

﻿                           The Great Gatsby                                  by                          F. Scott Fitzgerald                           Table of ContentsIIIIIIIVVVIVIIVIIIIX                              Once again                                  to                                 Zelda  Then wear the gold hat, if that will move her;  If you can bounce high, bounce for her too,  Till she cry “Lover, gold-hatted, high-bouncing lover,  I must have you!”  Thomas Parke d’Invilliers                                  IIn my younger and more vulnerable years my father gave me some advicethat I’ve been turning over in my mind ever since.“Whenever you feel like criticizing anyone,” he told me, “justremember that all the people in this world haven’t had the advantagesthat you’ve had.”He didn’t say any more, but we’ve always been unusually communicativein a reserved way, and I understood that he meant a great deal morethan that. In consequence, I’m inclined to reserve all judgement

In [4]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([87,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 41, 57, 54,  0, 28, 67, 54, 50,
        69,  0, 28, 50, 69, 68, 51, 74,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, 51, 74,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


## Splitting Data for Training and Validation


In [5]:
split_frac = 0.9
n = int(split_frac * len(data))
train_data = data[:n]
val_data = data[n:]

## Batching

In [7]:
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i + block_size] for i in ix])
  y = torch.stack([data[i+1: i+block_size+1] for i in ix])
  x,y = x.to(device), y.to(device)
  return x,y

x,y = get_batch('train')

## Context Manger that Disables Gradient Calculation:
### Infering without Gradient Calculation to speed up computation and improve memory usage.

In [8]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_interval)
    for k in range(eval_interval):
      X, Y = get_batch(split)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


## Head Class.
#### N Heads runnning in Parallel compute the KQ Dot Product, learning their own features which will subsequently be concatenated.

In [9]:
class Head(nn.Module):
  """one head of self attention"""
  def __init__(self, head_size):
    super().__init__()
    #each 384 to 96 features per head
    self.key = nn.Linear(n_embd, head_size, bias = False)
    self.query = nn.Linear(n_embd, head_size, bias = False)
    self.value = nn.Linear(n_embd, head_size, bias = False)
    #no lookahead, register to model state to prevent repeated overhead computation
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    #input of size (batch, time_step, channels)
    #output of size (batch, time-step, head_size)

    B,T,C = x.shape
    k = self.key(x) #(B,T,hs)
    q = self.query(x) #(B, T, hs)

    #compute attention scores ("affinities"), (-2,-1) flips second last column and last column
    #k.shape[-1]**-0.5 is the scaling factor, allows us to hear all parts of the conversation within each head evenly
    wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #for each value not exposed yet, fill with -inf instead
    wei = F.softmax(wei, dim = -1) #(B,T,T)
    wei = self.dropout(wei)
    #perform the weighted aggregation of the values
    v = self.value(x)
    out = wei @ v #(B,T,T) x (B,T, hs)
    return out


## Multi Head Attention Class
#### Initializes the Heads and Concatenates their output.

In [10]:
class MultiHeadAttention(nn.Module):
  #Multiple Heads of self-attention in parallel

  def __init__(self, num_heads, head_size):
    super().__init__()
    #ModuleList initializes heads in parallel, unlike nn.Sequential
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(head_size * num_heads, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim = -1)#(B,T,F) -> (B,T, (h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3, h4, h4, h4, h4))
    out = self.dropout(self.proj(out))
    return out

## FeedForward Class
#### Straightforward Neural Network Architecture

In [11]:
class FeedForward(nn.Module):
  #a simple linear layer followed by a non-linearity

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(

        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout),
        )
  def forward(self, x):
    return self.net(x)

## Decoder Block Class
#### Source: "Attention is all You Need": https://arxiv.org/abs/1706.03762
#### The GPT architecture drops the encoder block from the Transformer architecture, and uses a series of decoder blocks.

In [12]:
#Decoder Block Class

class Block(nn.Module):
  #Transformer block: communication followed by computation
  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head #how many features is each head in multi-head capturing
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    y = self.sa(x)
    x = self.ln1(x + y)
    y = self.ffwd(x)
    x = self.ln2(x + y)
    return x


## GPT Language Model Class:
#### Class that Initializes the weights, and processes the data through multiple decoder blocks, each with heads initialized and running in parallel.

In [13]:
class GPTLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd) #final layer norm
    self.lm_head = nn.Linear(n_embd, vocab_size)
    self.apply(self.init_weights)

  def init_weights(self, module):
    if isinstance(module, nn.Linear):
      torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
      torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)

  def forward(self, index, targets = None):
    B,T = index.shape
    tok_emb = self.token_embedding_table(index)
    pos_emb = self.position_embedding_table(torch.arange(T,device = device))
    x = tok_emb + pos_emb #look up broadcasting tensors
    x = self.blocks(x) #(B,T,C)
    x = self.ln_f(x)  #(B,T,C)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
      #B is batches, T is the number of tokens we're getting the embeddings for, and C is the embedding,i.e, one row in the embedding matrix
      #-corresponding to the character, i.e, the probabilities of
      B, T, C = logits.shape
      # print(f'B is {B}, T is {T}, C is {C}')

      #To deal with pytorch shape requirements, we reshape to have shape (B,T,C) transform into (B*T,C)
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    #index is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      #get the predictions
      logits, loss = self.forward(index)
      # print(f'index is {index}')
      # print(f'loss is {loss}')
      # print(f'shape of logits is {logits.shape}')
      #focus only on the last time step
      logits = logits[:,-1,:] #becomes (B,C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim = -1) #(B,C)
      #sample from the distribution
      index_next = torch.multinomial(probs, num_samples=1)
      #append sample index to running sequence
      index = torch.cat((index, index_next), dim = 1)
    return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)


In [ ]:
# context = torch.zeros((1,1), dtype = torch.long, device = device)
# generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())

## Loss Optimization and Inference

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step: {iter}, train_loss: {losses['train']:.3f}, val_loss: {losses['val']:.3f}")

  #sample a batch of data
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = model.forward(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
print(loss.item())

step: 0, train_loss: 4.599, val_loss: 4.595
step: 100, train_loss: 2.414, val_loss: 2.427
step: 200, train_loss: 2.079, val_loss: 2.081
step: 300, train_loss: 1.888, val_loss: 1.905
step: 400, train_loss: 1.766, val_loss: 1.800
step: 500, train_loss: 1.682, val_loss: 1.733
step: 600, train_loss: 1.608, val_loss: 1.686
step: 700, train_loss: 1.554, val_loss: 1.648
step: 800, train_loss: 1.507, val_loss: 1.623
step: 900, train_loss: 1.458, val_loss: 1.592
step: 1000, train_loss: 1.425, val_loss: 1.582
step: 1100, train_loss: 1.390, val_loss: 1.555
step: 1200, train_loss: 1.358, val_loss: 1.553
step: 1300, train_loss: 1.331, val_loss: 1.545
step: 1400, train_loss: 1.296, val_loss: 1.532
step: 1500, train_loss: 1.274, val_loss: 1.525
step: 1600, train_loss: 1.248, val_loss: 1.524
step: 1700, train_loss: 1.223, val_loss: 1.535
step: 1800, train_loss: 1.196, val_loss: 1.516
step: 1900, train_loss: 1.176, val_loss: 1.520
step: 2000, train_loss: 1.151, val_loss: 1.526
step: 2100, train_loss: 1

In [ ]:
print(f"Model device: {next(model.parameters()).device}")


In [ ]:
print(device)

In [ ]:
## Next Steps:
### Pre-Train the GPT on OpenWebTextCorpus.